# Building Autoencoder in Pytorch
* 참고: https://vaibhaw-vipul.medium.com/building-autoencoder-in-pytorch-34052d1d280c

* Autoencoder를 만들기 위해서는:
    * encoding function, decoding function, loss function(distance function의 역할)

In [1]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image

* CIFAR-10 데이터셋 다운로드 후 두가지 transformation을 진행한다.
    * ToTensor(): [0, 255] 사이의 값을 가지는 PIL Image나 numpy.ndarray(H\*W\*C)를 [0.0, 1.0] 사이의 값을 가지는 torch.FloatTensor로 바꿔준다.
    * Normalize(): mean & standard deviation을 통해 tensor image를 정규화한다.
* transformation을 적용하고 나면 (-2, 2) 사이의 값을 가지게 된다.

In [7]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(0.4914, 0.4822, 0.4466), (0.247, 0.243, 0.261)])
trainTransform = tv.transforms.Compose([tv.transforms.ToTensor(), tv.transforms.Normalize((0.4914, 0.4822, 0.4466), (0.247, 0.243, 0.261))])

trainset = tv.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

dataloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=False)
testset = tv.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
"""
Write model
"""

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder,self).__init__()
        
        self.encoder = nn.Sequential(
        nn.Conv2d(3, 6, kernel_size=5),
        nn.ReLU(True),
        nn.Conv2d(6, 16, kernel_size=5),
        nn.ReLU(True))
        
        self.decoder = nn.Sequential(
        nn.ConvTranspose2d(16, 6, kernel_size=5), 
        nn.ReLU(True), 
        nn.ConvTranspose2d(6, 3, kernel_size=5), 
        nn.ReLU(True))
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        
        return x

In [9]:
num_epochs = 5
batch_size = 2

In [10]:
model = Autoencoder().cpu()
distance = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), weight_decay=1e-5)

In [11]:
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = Variable(img).cpu()
        # ===================forward=====================
        output = model(img)
        loss = distance(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.data()))

TypeError: 'tuple' object is not callable